# Snake identification challenge 

Using transfer learning as in Activity 7 to start with. See full explanation for snake identification challenge [here](https://www.aicrowd.com/challenges/snake-species-identification-challenge#task).

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

if not torch.cuda.is_available():
    raise Exception("You should enagle GPU in the Runtime menu")
device = torch.device("cuda")

/usr/local/lib/python3.6/dist-packages/torchvision/io/_video_opt.py:17: UserWarning: video reader based on ffmpeg c++ ops not available
  warnings.warn("video reader based on ffmpeg c++ ops not available")


In [2]:
import os, shutil
import os.path

In [3]:
import pandas as pd

In [4]:
from PIL import Image

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from torchvision import datasets, transforms

In [7]:
# import skimage.io

## Data

In [8]:
# The path to the directory where the original
# dataset was uncompressed
dataset_dir = 'Images/' # '/datalab/train/'

train_dir = 'Images/train_images/'

df_train_labels = pd.read_csv(dataset_dir + 'train_labels.csv')

dictionary = pd.factorize(df_train_labels.scientific_name)

df_train_labels['labels'] = dictionary[0]

dict_labels = df_train_labels[['scientific_name', 'labels']].drop_duplicates()

# str(train_dir) + str(df_train_labels.hashed_id[0]) + ".jpg"

In [9]:
# test_images_small
# test_metadata_small.csv
challenge_dir = 'Images/test_images_small/'

df_challenge_labels = pd.read_csv(dataset_dir + 'test_metadata_small.csv') # NO LABELS IN CHALLENGE DATA SET!

#pd.merge(df_challenge_labels, dict_labels, left_on=['scientific_name'], right_on=['scientific_name'])


### Eliminating corrupted images from image list

In [10]:
if os.path.isfile(dataset_dir + 'train_labels_proc.csv'):
    # True
    # Loac csv file containing names and labels for non-corrupted images
    df = pd.read_csv(dataset_dir + 'train_labels_proc.csv')
else:
    # False
    # Create csv file containing names and labels for non-corrupted images
    df = df_train_labels.copy()

    for item in df.hashed_id:
        try:
            # myimage= Image.open(str(train_dir_ori) + str(df_train_labels.hashed_id[0]) + ".jpg")
            myimage= Image.open(str(train_dir) + str(item) + ".jpg") # Alternativa Eva: mirar el tamany de la imatge i x
        except IOError:                                              # filtrar aquelles que siguin < XX MB.
            print ("cannot identify image file", myimage)
            df = df[df.hashed_id != item]


    df.to_csv(dataset_dir + 'train_labels_proc.csv') # KEEP THIS ONE AND LOAD IT (do not execute preprocessing twice!)

In [11]:
if os.path.isfile(challenge_dir + 'test_metadata_small.csv'):
    # True
    # Loac csv file containing names and labels for non-corrupted images
    df_challenge = pd.read_csv(challenge_dir + 'test_metadata_small.csv')
else:
    # False
    # Create csv file containing names and labels for non-corrupted images
    df_challenge = df_challenge_labels.copy()

    for item in df_challenge.hashed_id:
        try:
            # myimage= Image.open(str(train_dir_ori) + str(df_train_labels.hashed_id[0]) + ".jpg")
            myimage= Image.open(str(challenge_dir) + str(item) + ".jpg") # Alternativa Eva: mirar el tamany de la imatge i x
        except IOError:                                              # filtrar aquelles que siguin < XX MB.
            print ("cannot identify image file", myimage)
            df_challenge = df_challenge[df_challenge.hashed_id != item]


    df_challenge.to_csv(dataset_dir + 'challenge_labels_proc.csv') # 

### Splitting data into train, test, and validation data sets

There's a good DataLoader example [here!!!](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).

In [12]:
# X = df.hashed_id[0:1000] # For code testing purposes only!
# y = df.labels[0:1000]
X = df.hashed_id
y = df.labels

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) # test_size=0.2

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) # test_size=0.2

In [14]:
# split train + val + test (random 70-20-10)
# df['hashed_id'].sample(n=3, random_state = 1)
# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
# train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.01, random_state=1)

In [15]:
len(X_train)

103822

In [16]:
len(X_test)

25696

In [18]:
len(X_val)

260

### Writing the dataloader

In [ ]:
class SnakeDataset():
    """Face Landmarks dataset."""

    def __init__(self, X_partN, y_partN, root_dir, transform=None):
        """
        Args:
            X_partN (array): Array with all image names for a given data set (train, test, or val).
            y_partN (array): Array with the corresponding labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.DataFrame({'x':X_partN, 'y':y_partN}) # X_train, y_train
        self.root_dir = root_dir # path imatge al disc 'train_dir'
        self.transform = transform # transformacions a aplicar

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 0]+'.jpg')
        # BUG found for skimage: https://github.com/numpy/numpy/issues/12744
        # image = io.imread(img_name)
        image = Image.open(img_name) # Image.open(FILENAME).convert('L')
        image = image.convert('RGB')
        labels = self.data.iloc[idx, 1]
        sample = {'image': image, 'labels': labels}

        if self.transform:
            # Transform to tensors with pytorch
            sample['image'] = self.transform(sample['image'])
            # From https://discuss.pytorch.org/t/change-labels-in-data-loader/36823/5
            # sample['labels'] = torch.tensor(sample['labels']) 
            # From https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-long-but-got-scalar-type-float-when-using-crossentropyloss/30542/2
            # sample['labels'] = torch.tensor(sample['labels'], dtype=torch.long, device=device) 
            # From https://discuss.pytorch.org/t/expected-object-of-scalar-type-long-but-got-scalar-type-float-for-argument-2-target/33102
            # sample['labels'] = torch.tensor(sample['labels'], dtype=torch.int64, device=device) 
            sample['labels'] = torch.tensor(sample['labels'], dtype=torch.float, device=device) 
            
                
        return sample
    
    # IF DATASET = EVAL => RETURN SAMPLE, IMAGE_PATH (EVA)

**TODO!!!!** Check whether challenge images are part of training data set AND REMOVE THEM IF SO!!!

In [ ]:
df.columns

In [ ]:
df_challenge.columns

In [ ]:
# We have to transform input images to parameters used to trained VGG16!!!
# https://medium.com/datadriveninvestor/creating-a-pytorch-image-classifier-da9db139ba80
transform = transforms.Compose([# size must be >= 224 
                                transforms.Resize(256), 
                                transforms.CenterCrop(224), 
                                transforms.ToTensor(), # Convert the image to a tensor with pixels in the range [0, 1]
                                # https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with Imagenet parameters
                                ])

In [ ]:
trainset = SnakeDataset(X_train, y_train, root_dir = train_dir, transform=transform)
testset = SnakeDataset(X_test, y_test, root_dir = train_dir, transform=transform)
valset = SnakeDataset(X_val, y_val, root_dir = train_dir, transform=transform)

In [ ]:
# print the sizes of first 4 samples and showing the images
fig = plt.figure()

for i in range(len(trainset)):
    sample = trainset[i]

    print(i, sample['image'].shape, sample['labels'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    plt.imshow(Image.open(os.path.join(train_dir, X_train.iloc[i]+'.jpg')))
    print("Labelled as: " + str(y_train.iloc[i]))

    if i == 3:
        plt.show()
        break

In [ ]:
# Let's define some hyper-parameters
hparams = {
    'batch_size': 100, #  40, 
    'num_epochs':12,
    'val_batch_size': 100, #  40, 
    'learning_rate':1e-3,
    'log_interval':100,
}

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
hparams['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=hparams['batch_size'], 
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    testset,
    batch_size=hparams['val_batch_size'], 
    shuffle=False)

eval_loader = torch.utils.data.DataLoader(
    valset,
    batch_size=hparams['val_batch_size'], 
    shuffle=False)

In [ ]:
# Similarly, we can sample a BATCH from the dataloader by running over its iterator
bimg, blabel = next(iter(train_loader))['image'], next(iter(train_loader))['labels']
print('Batch Img shape: ', bimg.shape)
print('Batch Label shape: ', blabel.shape)
print('The Batched tensors return a collection of {} RGB images ({} channel, {} height pixels, {} width pixels)'.format(bimg.shape[0],
                                                                                                                        bimg.shape[1],
                                                                                                                        bimg.shape[2],
                                                                                                                        bimg.shape[3]))
print('In the case of the labels, we obtain {} batched integers, one per image'.format(blabel.shape[0]))

Using a pre-trained VGG16 network.

In [ ]:
from torchvision.models import vgg16

pretrained_model = vgg16(pretrained=True)
pretrained_model.eval()
pretrained_model.to(device)

In [ ]:
feature_extractor = pretrained_model.features
feature_extractor

We will start by simply running instances of the previously-introduced ImageFolder Dataset to extract features from these images.

In [ ]:
def extract_features(dataset, loader, batch_size): # dataset = trainset, testset, valset; loader = train_loader, test_loader, val_loader
                                                   # batch_size = hparams['batch_size']
    features = np.zeros(shape=(len(dataset), 512, 7, 7)) # 4, 4
    labels = np.zeros(shape=(len(dataset),))
    with torch.no_grad():
        for i, sample_batched in enumerate(loader):
            # print(i)
            inputs_batch = sample_batched['image']
            labels_batch = sample_batched['labels']
            inputs_batch, labels_batch = inputs_batch.to(device), labels_batch.to(device)
            # print("Entering feature_extractor function")
            features_batch = feature_extractor(inputs_batch)
            # print(i * batch_size)
            # print((i + 1) * batch_size)
            # print(features_batch.cpu().detach().numpy().shape)
            features[i * batch_size : (i + 1) * batch_size] = features_batch.cpu().detach().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch.cpu().detach().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            #features[i * batch_size : (i + 1) * batch_size] = features_batch.cpu().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            #labels[i * batch_size : (i + 1) * batch_size] = labels_batch.cpu().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
    return features, labels

In [ ]:
train_features, train_labels = extract_features(trainset, train_loader, hparams['batch_size'])

In [ ]:
test_features, test_labels = extract_features(testset, test_loader, hparams['val_batch_size'])

In [ ]:
validation_features, validation_labels = extract_features(valset, eval_loader, hparams['val_batch_size'])

The extracted features are currently of shape (samples, 512, 4, 4). We will feed them to a densely-connected classifier, so first we must flatten them to (samples, 8192):

In [ ]:
train_features = np.reshape(train_features, (-1, 7 * 7 * 512)) # 4 * 4
test_features = np.reshape(test_features, (-1, 7 * 7 * 512)) # 4 * 4
validation_features = np.reshape(validation_features, (-1, 7 * 7 * 512)) # 4 * 4

Adding the classifier top layers.

In [ ]:
len(dict_labels)

In [ ]:
# We have a total of 'len(dict_labels)' different labels to classify your images into. 
# MULTIPLE CLASSIFICATOR!

feature_classifier = nn.Sequential(
        nn.Linear(7*7*512, 256), # nn.Linear(8 * 8 * 64, 512), # 4*4*
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256,len(dict_labels)), # Number of different labels to classify our images into
        nn.LogSoftmax(dim=-1) # From Activitat 6
)

feature_classifier.to(device)

In [ ]:
optimizer = optim.Adam(feature_classifier.parameters(), lr=0.001)

# As we are dealing here with a multiple class problem, we need to use torch.nn.CrossEntropyLoss() as the loss
# function (see https://stackoverflow.com/questions/56821729/pytorch-bceloss-valueerror-target-and-input-must-have-the-same-number-of-ele).
# loss_fn = nn.BCELoss()
loss_fn = nn.CrossEntropyLoss()

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def train_model(model, optimizer, loss_fn, train_loader, val_loader, epochs):

    train_accuracies, train_losses, val_accuracies, val_losses = [], [], [], []
    val_loss = AverageMeter()
    val_accuracy = AverageMeter()
    train_loss = AverageMeter()
    train_accuracy = AverageMeter()

    for epoch in range(epochs):
        # train
        model.train()
        train_loss.reset()
        train_accuracy.reset()
        train_loop = tqdm(train_loader, unit=" batches")  # For printing the progress bar
        for data, target in train_loop:
            train_loop.set_description('[TRAIN] Epoch {}/{}'.format(epoch + 1, epochs))
            data, target = data.float().to(device), target.float().to(device)
            target = target.unsqueeze(-1)
            
            optimizer.zero_grad()
            
            output = model(data)
            target = target.squeeze(1)
            
            loss = loss_fn(output.double(), target.long())
            
            loss.backward()
            optimizer.step()

            train_loss.update(loss.item(), n=len(target))
            
            pred = output.round()  # get the prediction
            
            class_pred=torch.argmax(pred, dim=1)
            
            acc = (class_pred==target).sum().item()/len(target)
            
            train_accuracy.update(acc, n=len(target))
            train_loop.set_postfix(loss=train_loss.avg, accuracy=train_accuracy.avg)

        train_losses.append(train_loss.avg)
        train_accuracies.append(train_accuracy.avg)
        
        # validation
        model.eval()
        val_loss.reset()
        val_accuracy.reset()
        val_loop = tqdm(val_loader, unit=" batches")  # For printing the progress bar
        with torch.no_grad():
            for data, target in val_loop:
                val_loop.set_description('[VAL] Epoch {}/{}'.format(epoch + 1, epochs))
                data, target = data.float().to(device), target.float().to(device)
                target = target.unsqueeze(-1)
                #target = target.squeeze(1)
                output = model(data)
                target = target.squeeze(1)
                
                loss = loss_fn(output.double(), target.long())
                val_loss.update(loss.item(), n=len(target))
                
                pred = output.round()  # get the prediction
                #acc = pred.eq(target.view_as(pred)).sum().item()/len(target)
                class_pred=torch.argmax(pred, dim=1)
                acc = (class_pred==target).sum().item()/len(target)
                #acc = pred.eq(target.view_as(pred)).sum().item()/len(target)
                val_accuracy.update(acc, n=len(target))
                val_loop.set_postfix(loss=val_loss.avg, accuracy=val_accuracy.avg)

        val_losses.append(val_loss.avg)
        val_accuracies.append(val_accuracy.avg)
        
    return train_accuracies, train_losses, val_accuracies, val_losses

In [ ]:
from torch.utils.data import TensorDataset

train_features_dataset = TensorDataset(torch.tensor(train_features), torch.tensor(train_labels, dtype=torch.float, device=device)) # dtype=torch.long
train_features_loader = DataLoader(train_features_dataset, batch_size=hparams['batch_size'], shuffle=True)

test_features_dataset = TensorDataset(torch.tensor(test_features), torch.tensor(test_labels, dtype=torch.float64, device=device))
test_features_loader = DataLoader(test_features_dataset, batch_size=hparams['val_batch_size'], shuffle=False)

val_features_dataset = TensorDataset(torch.tensor(validation_features), torch.tensor(validation_labels, dtype=torch.float64, device=device))
val_features_loader = DataLoader(val_features_dataset, batch_size=hparams['val_batch_size'], shuffle=False)

In [ ]:
train_accuracies, train_losses, val_accuracies, val_losses = train_model(feature_classifier, optimizer, loss_fn, 
                                                                         train_features_loader, val_features_loader, 
                                                                         hparams['num_epochs'])

In [ ]:
epochs = range(len(train_accuracies))

plt.plot(epochs, train_accuracies, 'b', label='Training acc')
plt.plot(epochs, val_accuracies, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, train_losses, 'b', label='Training loss')
plt.plot(epochs, val_losses, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

**Check [this](https://heartbeat.fritz.ai/basics-of-image-classification-with-pytorch-2f8973c51864)!!! It's training beautifully explained!!!**

In [ ]:
class SnakeDataset_challenge():
    """Face Landmarks dataset."""

    def __init__(self, X_partN, root_dir, transform=None):
        """
        Args:
            X_partN (array): Array with all image names for a given data set (train, test, or val).
            y_partN (array): Array with the corresponding labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.DataFrame(X_partN) # X_train, y_train
        self.root_dir = root_dir # path imatge al disc 'train_dir'
        self.transform = transform # transformacions a aplicar

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 0]+'.jpg')
        # BUG found for skimage: https://github.com/numpy/numpy/issues/12744
        # image = io.imread(img_name)
        image = Image.open(img_name) # Image.open(FILENAME).convert('L')
        image = image.convert('RGB')
        #labels = self.data.iloc[idx, 1]
        # sample = image
        sample = {'image': image, 'hashed_id': self.data.iloc[idx, 0]}

        if self.transform:
            # Transform to tensors with pytorch
            sample['image'] = self.transform(sample['image'])
            # sample['labels'] = torch.tensor(sample['labels'], dtype=torch.float, device=device) 
            
        return sample #, self.data.iloc[idx, 0]
    
    # IF DATASET = EVAL => RETURN SAMPLE, IMAGE_PATH (EVA)

In [ ]:
#save model
# torch.save(feature_classifier.state_dict(), "first_model.pt")
# np.savez('history_first_model.npz', train_accuracies=train_accuracies, train_losses=train_losses, val_accuracies=val_accuracies, val_losses=val_losses)
torch.save(feature_classifier.state_dict(), "first_model_v2.pt")
np.savez('history_first_model_v2.npz', train_accuracies=train_accuracies, train_losses=train_losses, val_accuracies=val_accuracies, val_losses=val_losses)

In [ ]:
#load model
model = feature_classifier #classificador
model.load_state_dict(torch.load("first_model_v2.pt"))

In [ ]:
# Transform challenge data set
# SnakeDataset_challenge
X_challenge = df_challenge.hashed_id
challengeset = SnakeDataset_challenge(X_challenge, root_dir = challenge_dir, transform=transform)

In [ ]:
challengeset

In [ ]:
img, hashedid = challengeset[0]['image'], challengeset[0]['hashed_id']
print('Img shape: ', img.shape)
print('hashedid: ', hashedid)

In [ ]:
challenge_loader = torch.utils.data.DataLoader(
    challengeset,
    batch_size=200, 
    shuffle=False)

In [ ]:
def challenge_extract_features(dataset, loader, batch_size): # dataset = trainset, testset, valset; loader = train_loader, test_loader, val_loader
                                                   # batch_size = hparams['batch_size']
    features = np.zeros(shape=(len(dataset), 512, 7, 7)) # 4, 4
    # labels = np.zeros(shape=(len(dataset),))
    hashedids = [] #np.zeros(shape=(len(dataset),))
    with torch.no_grad():
        for i, sample_batched in enumerate(loader):
            # print(i)
            inputs_batch = sample_batched['image']
            #labels_batch = sample_batched['labels']
            hashedids_batch = sample_batched['hashed_id']
            # inputs_batch, labels_batch = inputs_batch.to(device), labels_batch.to(device)
            inputs_batch = inputs_batch.to(device)
            # print("Entering feature_extractor function")
            features_batch = feature_extractor(inputs_batch)
            features[i * batch_size : (i + 1) * batch_size] = features_batch.cpu().detach().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            # labels[i * batch_size : (i + 1) * batch_size] = labels_batch.cpu().detach().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            # hashedids[i * batch_size : (i + 1) * batch_size] = hashedids_batch#.cpu().detach().numpy() #hashedids_batch.cpu().detach().numpy()
            hashedids.extend(hashedids_batch)
            
    return features, hashedids #, labels

In [ ]:
challenge_features, challenge_hashedids = challenge_extract_features(challengeset, challenge_loader, 200)

In [ ]:
# Feed classifier with challenge_features
challenge_features = np.reshape(challenge_features, (-1, 7 * 7 * 512)) # 4 * 4

In [ ]:
challenge_output = feature_classifier(torch.tensor(challenge_features, dtype=torch.float, device=device))

As we are using `nn.LogSoftmax()` in the final layer of the classifier, and given that `LogSoftmax(x_i) = log(Softmax(x_i))`, and `Softmax(dim=-1)` gives the probabilities of belonging to considered classes for a given input (`x_i`) normalized to 1. If we want to recover the probabilities normalized to 1, we just need to do: `probability(x_i) = exp(LogSoftmax(x_i))`

In [ ]:
challenge_output 

In [ ]:
# From https://www.programcreek.com/python/example/101149/torch.exp
torch.exp(challenge_output)

In [ ]:
# Get probabilities normalized to 1
challenge_probabilities = torch.exp(challenge_output)
challenge_probabilities.shape

In [ ]:
# Get most probable class with torch.argmax
challenge_pred = challenge_output.round()  # get the prediction
challenge_class_pred=torch.argmax(challenge_pred, dim=1)

In [ ]:
challenge_class_pred

### Saving output to a .csv file

In [ ]:
# challenge_class_pred -> to name of snake (use dictionary)
# challenge_probabilities
# hashed_id
#
# Input: challenge_features, challenge_hashedids
df_challenge['label_pred'] = challenge_class_pred.cpu().numpy()

In [ ]:
df_challenge = pd.merge(df_challenge, dict_labels, left_on=['label_pred'], right_on=['labels'])

In [ ]:
df_challenge = df_challenge.drop(columns=['labels'])

In [ ]:
df_challenge = df_challenge.rename(columns={"scientific_name": "scientific_name_pred"})

In [ ]:
prob_cols = list(range(0, 85))

In [ ]:
df_probabilities = pd.DataFrame(challenge_probabilities.cpu().detach().numpy(), columns=prob_cols) # , columns=['a', 'b', 'c'])


In [ ]:
df_challenge_output = pd.concat([df_challenge, df_probabilities], axis=1)

In [ ]:
df_challenge_output.head()

In [ ]:
df_challenge_output.to_csv(dataset_dir + 'challenge_output_v2.csv', index=False)

In [ ]:
df_challenge = pd.merge(df_challenge, dict_labels, left_on=['label_pred'], right_on=['labels'])

In [ ]:
df_prob_cols = pd.DataFrame(prob_cols, columns=['labels'])

In [ ]:
df_prob_cols = pd.merge(df_prob_cols, dict_labels, left_on=['labels'], right_on=['labels'])

In [ ]:
df_probabilities_names = pd.DataFrame(challenge_probabilities.cpu().detach().numpy(), columns=df_prob_cols['scientific_name'].values)

In [ ]:
df_challenge_output_names = pd.concat([df_challenge, df_probabilities_names], axis=1)

In [ ]:
df_challenge_output_names.head()

In [ ]:
df_challenge_output_names.to_csv(dataset_dir + 'challenge_output_names_v2.csv', index=False)

## Fine tuning the last 3 convolutional layers

In [ ]:
for layer in feature_extractor[:24]:  # Freeze layers 0 to 23
    for param in layer.parameters():
        param.requires_grad = False

for layer in feature_extractor[24:]:  # Train layers 24 to 30
    for param in layer.parameters():
        param.requires_grad = True

In [ ]:
model = nn.Sequential(
        feature_extractor,
        nn.Flatten(),
        feature_classifier
)

model.to(device)

In [ ]:
def train_model_ft(model, optimizer, loss_fn, train_loader, val_loader, epochs):

    train_accuracies, train_losses, val_accuracies, val_losses = [], [], [], []
    val_loss = AverageMeter()
    val_accuracy = AverageMeter()
    train_loss = AverageMeter()
    train_accuracy = AverageMeter()

    for epoch in range(epochs):
        # train
        model.train()
        train_loss.reset()
        train_accuracy.reset()
        train_loop = tqdm(train_loader, unit=" batches")  # For printing the progress bar
        # for data, target in train_loop:
        for item in train_loop:
            train_loop.set_description('[TRAIN] Epoch {}/{}'.format(epoch + 1, epochs))
            data, target = item['image'].float().to(device), item['labels'].float().to(device)
            target = target.unsqueeze(-1)
            
            optimizer.zero_grad()
            
            output = model(data)
            target = target.squeeze(1)
            
            loss = loss_fn(output.double(), target.long())
            
            loss.backward()
            optimizer.step()

            train_loss.update(loss.item(), n=len(target))
            
            pred = output.round()  # get the prediction
            
            class_pred=torch.argmax(pred, dim=1)
            
            acc = (class_pred==target).sum().item()/len(target)
            
            train_accuracy.update(acc, n=len(target))
            train_loop.set_postfix(loss=train_loss.avg, accuracy=train_accuracy.avg)

        train_losses.append(train_loss.avg)
        train_accuracies.append(train_accuracy.avg)
        
        # validation
        model.eval()
        val_loss.reset()
        val_accuracy.reset()
        val_loop = tqdm(val_loader, unit=" batches")  # For printing the progress bar
        with torch.no_grad():
            for item in val_loop:
                val_loop.set_description('[VAL] Epoch {}/{}'.format(epoch + 1, epochs))
                data, target = item['image'].float().to(device), item['labels'].float().to(device)
                target = target.unsqueeze(-1)
                output = model(data)
                target = target.squeeze(1)
                
                loss = loss_fn(output.double(), target.long())
                val_loss.update(loss.item(), n=len(target))
                
                pred = output.round()  # get the prediction
                #acc = pred.eq(target.view_as(pred)).sum().item()/len(target)
                class_pred=torch.argmax(pred, dim=1)
                acc = (class_pred==target).sum().item()/len(target)
                #acc = pred.eq(target.view_as(pred)).sum().item()/len(target)
                val_accuracy.update(acc, n=len(target))
                val_loop.set_postfix(loss=val_loss.avg, accuracy=val_accuracy.avg)

        val_losses.append(val_loss.avg)
        val_accuracies.append(val_accuracy.avg)
        
    return train_accuracies, train_losses, val_accuracies, val_losses

In [ ]:
train_accuracies_ft, train_losses_ft, test_accuracies_ft, test_losses_ft = train_model_ft(model, optimizer, loss_fn,
                                                                                          train_loader, test_loader,
                                                                                          hparams['num_epochs'])

In [ ]:
# torch.save(model.state_dict(), "second_model.pt")
# np.savez('history_second_model.npz', train_accuracies=train_accuracies_ft, train_losses=train_losses_ft, 
#          val_accuracies=test_accuracies_ft, val_losses=test_losses_ft)
torch.save(model.state_dict(), "second_model_v2.pt")
np.savez('history_second_model_v2.npz', train_accuracies=train_accuracies_ft, train_losses=train_losses_ft, 
         val_accuracies=test_accuracies_ft, val_losses=test_losses_ft)

In [ ]:
epochs = range(len(train_accuracies_ft))

plt.plot(epochs, train_accuracies_ft, 'b', label='Training acc')
plt.plot(epochs, test_accuracies_ft, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, train_losses_ft, 'b', label='Training loss')
plt.plot(epochs, test_losses_ft, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### Hold out metrics - PENDING!

In [19]:
class SnakeDataset():
    """Face Landmarks dataset."""

    def __init__(self, X_partN, y_partN, root_dir, transform=None):
        """
        Args:
            X_partN (array): Array with all image names for a given data set (train, test, or val).
            y_partN (array): Array with the corresponding labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.DataFrame({'x':X_partN, 'y':y_partN}) # X_train, y_train
        self.root_dir = root_dir # path imatge al disc 'train_dir'
        self.transform = transform # transformacions a aplicar

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 0]+'.jpg')
        # BUG found for skimage: https://github.com/numpy/numpy/issues/12744
        # image = io.imread(img_name)
        image = Image.open(img_name) # Image.open(FILENAME).convert('L')
        image = image.convert('RGB')
        labels = self.data.iloc[idx, 1]
        sample = {'image': image, 'labels': labels}

        if self.transform:
            # Transform to tensors with pytorch
            sample['image'] = self.transform(sample['image'])
            # From https://discuss.pytorch.org/t/change-labels-in-data-loader/36823/5
            # sample['labels'] = torch.tensor(sample['labels']) 
            # From https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-long-but-got-scalar-type-float-when-using-crossentropyloss/30542/2
            # sample['labels'] = torch.tensor(sample['labels'], dtype=torch.long, device=device) 
            # From https://discuss.pytorch.org/t/expected-object-of-scalar-type-long-but-got-scalar-type-float-for-argument-2-target/33102
            # sample['labels'] = torch.tensor(sample['labels'], dtype=torch.int64, device=device) 
            sample['labels'] = torch.tensor(sample['labels'], dtype=torch.float, device=device) 
            
                
        return sample
    
    # IF DATASET = EVAL => RETURN SAMPLE, IMAGE_PATH (EVA)

In [20]:
# We have to transform input images to parameters used to trained VGG16!!!
# https://medium.com/datadriveninvestor/creating-a-pytorch-image-classifier-da9db139ba80
transform = transforms.Compose([# size must be >= 224 
                                transforms.Resize(256), 
                                transforms.CenterCrop(224), 
                                transforms.ToTensor(), # Convert the image to a tensor with pixels in the range [0, 1]
                                # https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with Imagenet parameters
                                ])

In [21]:
valset = SnakeDataset(X_val, y_val, root_dir = train_dir, transform=transform)

In [22]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [23]:
len(X_val)

260

In [24]:
valset

In [25]:
eval_loader = torch.utils.data.DataLoader(
    valset,
    batch_size=len(X_val), #hparams['val_batch_size'], 
    shuffle=False)

In [26]:
eval_accuracies, eval_losses = [], []
eval_loss = AverageMeter()
eval_accuracy = AverageMeter()

In [27]:
from torchvision.models import vgg16
pretrained_model = vgg16(pretrained=True)

feature_extractor = pretrained_model.features
feature_classifier = nn.Sequential(
        nn.Linear(7*7*512, 256), # nn.Linear(8 * 8 * 64, 512), # 4*4*
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256,len(dict_labels)), # Number of different labels to classify our images into
        nn.LogSoftmax(dim=-1) # From Activitat 6
)

model = nn.Sequential(
        feature_extractor,
        nn.Flatten(),
        feature_classifier
)

model.to(device)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [28]:
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

#load model
model.load_state_dict(torch.load("second_model_v2.pt"))
# model = TempModel() # model = model()
#model.load_state_dict(torch.load('second_model.pt', map_location=map_location))
model.eval()

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [30]:
# optimizer = optim.Adam(feature_classifier.parameters(), lr=0.001)

# As we are dealing here with a multiple class problem, we need to use torch.nn.CrossEntropyLoss() as the loss
# function (see https://stackoverflow.com/questions/56821729/pytorch-bceloss-valueerror-target-and-input-must-have-the-same-number-of-ele).
# loss_fn = nn.BCELoss()
loss_fn = nn.CrossEntropyLoss()

In [31]:
# Let's define some hyper-parameters
hparams = {
    'batch_size': 100, #  40, 
    'num_epochs':12,
    'val_batch_size': 100, #  40, 
    'learning_rate':1e-3,
    'log_interval':100,
}

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
hparams['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

epoch = hparams['num_epochs']

In [33]:
#model.eval()
eval_loss.reset()
eval_accuracy.reset()
val_loop = tqdm(eval_loader, unit=" batches")  # For printing the progress bar
with torch.no_grad():
    # for data, target in val_loop:
    for item in val_loop:
        #val_loop.set_description('[VAL] Epoch {}/{}'.format(epoch + 1, epochs))
        #data, target = data.float().to(device), target.float().to(device)
        data, target = item['image'].float().to(device), item['labels'].float().to(device)
        target = target.unsqueeze(-1)
        #target = target.squeeze(1)
        output = model(data)
        target = target.squeeze(1)

        loss = loss_fn(output.double(), target.long())
        eval_loss.update(loss.item(), n=len(target))

        pred = output.round()  # get the prediction
        class_pred=torch.argmax(pred, dim=1)
        acc = (class_pred==target).sum().item()/len(target)
        eval_accuracy.update(acc, n=len(target))
        val_loop.set_postfix(loss=eval_loss.avg, accuracy=eval_accuracy.avg)

eval_losses.append(eval_loss.avg)
eval_accuracies.append(eval_accuracy.avg)

RuntimeError: CUDA out of memory. Tried to allocate 3.11 GiB (GPU 0; 11.17 GiB total capacity; 3.34 GiB already allocated; 2.18 GiB free; 8.69 GiB reserved in total by PyTorch)

In [ ]:
def challenge_extract_features(dataset, loader, batch_size): # dataset = trainset, testset, valset; loader = train_loader, test_loader, val_loader
                                                   # batch_size = number_of_images_to_predict!!!
    features = np.zeros(shape=(len(dataset), 512, 7, 7)) # 4, 4
    # labels = np.zeros(shape=(len(dataset),))
    hashedids = [] #np.zeros(shape=(len(dataset),))
    with torch.no_grad():
        for i, sample_batched in enumerate(loader):
            # print(i)
            inputs_batch = sample_batched['image']
            #labels_batch = sample_batched['labels']
            hashedids_batch = sample_batched['hashed_id']
            # inputs_batch, labels_batch = inputs_batch.to(device), labels_batch.to(device)
            inputs_batch = inputs_batch.to(device)
            # print("Entering feature_extractor function")
            features_batch = feature_extractor(inputs_batch)
            features[i * batch_size : (i + 1) * batch_size] = features_batch.cpu().detach().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            # labels[i * batch_size : (i + 1) * batch_size] = labels_batch.cpu().detach().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            # hashedids[i * batch_size : (i + 1) * batch_size] = hashedids_batch#.cpu().detach().numpy() #hashedids_batch.cpu().detach().numpy()
            hashedids.extend(hashedids_batch)
            
    return features, hashedids #, labels

### Confusion matrix - PENDING!!!

In [ ]:
df_pred = pd.DataFrame(challenge_class_pred.cpu().numpy(), columns=['label_pred'])

In [ ]:
# local_path = '/Users/marta/Dropbox/DataScience/DeepLearning_PQTM/UnsupervisedLearning/Activity_I/'

plot_confusion_matrix(cm           = confusion_matrix(df_pred['label_pred'], df_pred['label_pred']), 
                      normalize    = False,
                      target_names = ['acc', 'unacc'], # true_negative, true_positive
                      save_to_path = dataset_dir, # local_path,
                      fig_name = 'ConfusionMatrix_first_model.png',
                      title        = "Confusion Matrix")

In [ ]:
# JUST A COPY FROM PREVIOUS CELLS, YOU NEED TO BE CAREFUL WITH 
def extract_features(dataset, loader, batch_size): # dataset = trainset, testset, valset; loader = train_loader, test_loader, val_loader
                                                   # batch_size = hparams['batch_size']
    features = np.zeros(shape=(len(dataset), 512, 7, 7)) # 4, 4
    labels = np.zeros(shape=(len(dataset),))
    with torch.no_grad():
        for i, sample_batched in enumerate(loader):
            # print(i)
            inputs_batch = sample_batched['image']
            labels_batch = sample_batched['labels']
            inputs_batch, labels_batch = inputs_batch.to(device), labels_batch.to(device)
            # print("Entering feature_extractor function")
            features_batch = feature_extractor(inputs_batch)
            # print(i * batch_size)
            # print((i + 1) * batch_size)
            # print(features_batch.cpu().detach().numpy().shape)
            features[i * batch_size : (i + 1) * batch_size] = features_batch.cpu().detach().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch.cpu().detach().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            #features[i * batch_size : (i + 1) * batch_size] = features_batch.cpu().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
            #labels[i * batch_size : (i + 1) * batch_size] = labels_batch.cpu().numpy() # CAREFUL WITH .detach() HERE!!! NO GRADIENT WILL BE COMPUTED (so no backprop)!!!.numpy()
    return features, labels